### Classificador de noticias
 Esse é um projeto que visa por meio de um dataset com a compilação de noticias de diversos meios de comunicação e sendo categorizado pela veracidade das noticias. 

 Este data set esta no site do Kaggle :https://www.kaggle.com/jruvika/fake-news-detection

In [ ]:
# Importação de bibliotecas para utilização na Leitura e na manipulação de dados
import pandas as pd
import numpy as np

### Carregamento do Data Set

In [ ]:
# Carregamento pelo pandas por meio do read CSV
df_fake_news = pd.read_csv("../input/fake-news-detection/data.csv")

In [ ]:
df_fake_news.head(5) 

In [ ]:
# Analise da qualidade dos dados importados
df_fake_news.describe()

### Análise Descritiva
A principio seria analisado o corpo do texto(Body) do Data Frame. Como este possivelmente tem dados faltantes dentro da coluna, foi escolhido a manchete(Headline) para se usar como paramentro da analise.

In [ ]:
#Importação do Collection para observar melhor os dados que serão analisados
from collections import Counter

In [ ]:
Counter(df_fake_news["Label"])

In [ ]:
# Alterar os dados da coluna Label para melhor vizualização dos resultados
df_fake_news['Label'].replace(1, 'News',regex=True, inplace=True) 

In [ ]:
df_fake_news['Label'].replace(0, 'Fake News',regex=True, inplace=True)

In [ ]:
Counter(df_fake_news["Label"])

Apesar do numero maior de Fake News foi tomada decisão de continuar sem retirada de elementos do Data Frame, pois a diferença não me pareceu significante para aplicar um drop() em linhas relativas ao Fake News

### Limpeza de Dados
Esta parte é importante quando estamos trabalhando com o Naive Bayes para que ele possa ler os padrões e montar um modelo preditivo.

In [ ]:
#importação do NLTK para ajudar no processamento das informações para o Naive Bayes
import nltk
nltk.download('stopwords')

In [ ]:
# Importação das stopwords para fazer a limpeza da coluna Headline
from nltk.corpus import stopwords

In [ ]:
# Estabelecendo as stopwords em inglês
list_stops_words = stopwords.words("english")

In [ ]:
df_fake_news["Headline"] = df_fake_news['Headline'].str.lower()

In [ ]:
df_fake_news["Headline"].replace('\n',' ', regex=True, inplace=True)

### Separação em Treinamento e Teste
Agora com os dados já tratados temos que estabelecer as porcentagem de treino o nosso modelo de Maachine Learning

In [ ]:
#importação do sklearn para criar modelo de previsão
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_fake_news["Headline"], df_fake_news["Label"], 
                                                    test_size=0.30, 
                                                    random_state=42)

Foi estabelecido um treinamento com 70% dos dados e fazer testes com 30% dos dados. 

Esse treinamento é importante para que o algoritimo possa melhorar sua capacidade preditiva.

### Pipeline Model
Foram exportado o **Pipeline** para melhorar o entendimento do codigo, **Naive Bayes** na sua forma de multinomial, ***Count Vectorizer** que segmentas todas as palavras e conta as frequencia de cada uma e o **Tf Idf** Transformer que diminui o peso das palavras mais frequentes.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
classify = Pipeline(
                [('vect', CountVectorizer(stop_words= list_stops_words)), 
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB())
                 ])

In [ ]:
# treinamento do nosso modelo
classify.fit(X_train, y_train)

### Avaliação do Modelo
Agora temos nosso modelo treinado, medimos a capacidade de predição. Que consegue prever cerca de 83%. O Modelo também foi capaz de prever com precisão muito parecida tanto as Fake News quanto as Noticias verdadeira. Mas podemos ver que o modelo tem uma taxa baixa de recall nas Noticias Verdadeiras, que pode ser por conta da diferença amostral das duas variaveis.

In [ ]:
# Importação do metrics para medir o aprendizado da maquina
from sklearn import metrics

In [ ]:
# Medição da precisão do modelo utilizando as amostras
classify.score(X_test, y_test)

In [ ]:
# Capacidade de predição do modelo
preds = classify.predict(X_test)
print(metrics.classification_report(y_test, preds))

### Predição do Modelo
Agora colocando o modelo para predizer uma Manchete

In [ ]:
texto  = "Trump and his allies respond with pseudo-science as US death toll hits 150,000"

In [ ]:
# Testando a predição do modelo
classify.predict([texto])

In [ ]:
# Base da decisão pela classificação como News
classify.predict_proba([texto])